In [4]:
from __future__ import print_function, division

import math
import pandas as pd
import random
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import re

import chap01soln
import first
import linear
import thinkplot
import thinkstats2

In [5]:
live, firsts, others = first.MakeFrames()

nsfg.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.birthwgt_lb[df.birthwgt_lb > 20] = np.nan


In [3]:
formula = 'totalwgt_lb ~ agepreg'
model = smf.ols(formula, data=live)
results = model.fit()

In [4]:
results.params

Intercept    6.830397
agepreg      0.017454
dtype: float64

In [5]:
results.pvalues

Intercept    0.000000e+00
agepreg      5.722947e-11
dtype: float64

In [6]:
results.rsquared

0.0047381154747098142

In [7]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Thu, 19 Nov 2015   Prob (F-statistic):           5.72e-11
Time:                        12:11:44   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.0

In [3]:
def SummarizeResults(results):
    """Prints the most important parts of linear regression results:

    results: RegressionResults object
    """
    for name, param in results.params.iteritems():
        pvalue = results.pvalues[name]
        print('%s   %0.3g   (%.3g)' % (name, param, pvalue))

    try:
        print('R^2 %.4g' % results.rsquared)
        ys = results.model.endog
        print('Std(ys) %.4g' % ys.std())
        print('Std(res) %.4g' % results.resid.std())
    except AttributeError:
        print('R^2 %.4g' % results.prsquared)

In [9]:
SummarizeResults(results)

Intercept   6.83   (0)
agepreg   0.0175   (5.72e-11)
R^2 0.004738
Std(ys) 1.408
Std(res) 1.405


In [11]:
diff_weight = firsts.totalwgt_lb.mean() - others.totalwgt_lb.mean()
print(diff_weight)

-0.124761184535


In [12]:
diff_age = firsts.agepreg.mean() - others.agepreg.mean()
print(diff_age)

-3.58643476615


In [13]:
results = smf.ols('totalwgt_lb ~ agepreg', data=live).fit()
slope = results.params['agepreg']

In [14]:
slope * diff_age

-0.062597099721694888

In [15]:
live['isfirst'] = live.birthord == 1
formula = 'totalwgt_lb ~ isfirst'
results = smf.ols(formula, data=live).fit()
SummarizeResults(results)

Intercept   7.33   (0)
isfirst[T.True]   -0.125   (2.55e-05)
R^2 0.00196
Std(ys) 1.408
Std(res) 1.407


In [16]:
formula = 'totalwgt_lb ~ isfirst + agepreg'
results = smf.ols(formula, data=live).fit()
SummarizeResults(results)

Intercept   6.91   (0)
isfirst[T.True]   -0.0698   (0.0253)
agepreg   0.0154   (3.93e-08)
R^2 0.005289
Std(ys) 1.408
Std(res) 1.405


In [17]:
live['agepreg2'] = live.agepreg**2
formula = 'totalwgt_lb ~ isfirst + agepreg + agepreg2'
results = smf.ols(formula, data=live).fit()
SummarizeResults(results)

Intercept   5.69   (1.38e-86)
isfirst[T.True]   -0.0504   (0.109)
agepreg   0.112   (3.23e-07)
agepreg2   -0.00185   (8.8e-06)
R^2 0.007462
Std(ys) 1.408
Std(res) 1.403


In [3]:
live = live[live.prglngth > 30]
live['boy'] = (live.babysex == 1).astype(int)
resp = chap01soln.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

In [28]:
t = []
for name in join.columns:
    try:
        if join[name].var() < 1e-7 or name == 'caseid':
            continue
        
        formula = 'totalwgt_lb ~ agepreg + ' + name
        formula = formula.encode('ascii')
        model = smf.ols(formula, data=join)
        if model.nobs < len(join)/2:
            continue
        
        results = model.fit()
    except (ValueError, TypeError):
        continue
    
    t.append((results.rsquared, name))

In [29]:
t.sort(reverse=True)
for mse, name in t[:30]:
    print(name, mse)

totalwgt_lb 1.0
birthwgt_lb 0.949812730598
lbw1 0.300824078447
prglngth 0.130125194886
wksgest 0.123400413634
agecon 0.102031499282
mosgest 0.0271442746396
babysex 0.0185509252939
race_r 0.0161995035863
race 0.0161995035863
nbrnaliv 0.0160177527098
paydu 0.0140037955781
rmarout03 0.0134300664657
birthwgt_oz 0.0131024576157
anynurse 0.0125290225418
bfeedwks 0.0121936884045
totincr 0.0118700690312
marout03 0.0118078019944
marcon03 0.0117525993544
cebow 0.0114377709196
rmarout01 0.0114077371386
rmarout6 0.0113541384728
marout01 0.0112693572468
hisprace_r 0.011238349302
hisprace 0.011238349302
mar1diss 0.0109615635908
fmarcon5 0.0106049646843
rmarout02 0.0105469132066
marcon02 0.0104814017955
fmarout5 0.0104616913674


In [30]:
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 + '
           'nbrnaliv>1 + paydu==1 + totincr')
results = smf.ols(formula, data=join).fit()
SummarizeResults(results)

Intercept   6.63   (0)
C(race)[T.2]   0.357   (5.43e-29)
C(race)[T.3]   0.266   (2.33e-07)
babysex == 1[T.True]   0.295   (5.39e-29)
nbrnaliv > 1[T.True]   -1.38   (5.1e-37)
paydu == 1[T.True]   0.12   (0.000114)
agepreg   0.00741   (0.0035)
totincr   0.0122   (0.00188)
R^2 0.05999
Std(ys) 1.271
Std(res) 1.232


In [31]:
live['boy'] = (live.babysex == 1).astype(int)
model = smf.logit('boy ~ agepreg', data=live)
results = model.fit()
SummarizeResults(results)

Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 3
Intercept   0.00579   (0.953)
agepreg   0.00105   (0.783)
R^2 6.144e-06


In [32]:
results.pvalues

Intercept    0.952696
agepreg      0.783272
dtype: float64

In [33]:
formula = 'boy ~ agepreg + hpagelb + birthord + C(race)'
model = smf.logit(formula, data=live)
results = model.fit()
SummarizeResults(results)

Optimization terminated successfully.
         Current function value: 0.692944
         Iterations 3
Intercept   -0.0301   (0.772)
C(race)[T.2]   -0.0224   (0.66)
C(race)[T.3]   -0.000457   (0.996)
agepreg   -0.00267   (0.629)
hpagelb   0.0047   (0.266)
birthord   0.00501   (0.821)
R^2 0.000144


In [35]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names) 
actual = endog['boy']
actual.mean()

0.507173764518333

In [37]:
predict = (results.predict() >= 0.5)
true_pos = predict * actual
true_neg = (1-predict)*(1-actual)
print(true_pos, true_neg)

0       1
1       0
2       1
3       0
4       0
5       1
6       0
7       0
8       1
9       0
10      1
11      0
12      1
13      0
14      0
15      1
16      0
17      0
18      1
19      1
20      0
21      1
22      1
23      1
24      1
25      1
26      0
27      1
28      1
29      1
       ..
8752    1
8753    1
8754    1
8755    1
8756    0
8757    1
8758    0
8759    0
8760    1
8761    1
8762    0
8763    0
8764    0
8765    0
8766    0
8767    1
8768    0
8769    0
8770    1
8771    0
8772    0
8773    0
8774    1
8775    0
8776    1
8777    1
8778    0
8779    0
8780    1
8781    1
Name: boy, dtype: float64 0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
8752    0
8753    0
8754    0
8755    0
8756    0
8757

In [38]:
acc = (sum(true_pos) + sum(true_neg))/len(actual)
acc

0.51150079708494645

In [6]:
live = live[live.prglngth > 30]
live['boy'] = (live.babysex == 1).astype(int)
resp = chap01soln.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

In [7]:
t = []
formula = 'boy ~ '
variables = []
for name in join.columns:
    try:
        if join[name].var() < 1e-7 or name == 'finalwgt_r':
            continue
        formula_test = formula + name
        formula_test = formula_test.encode('ascii')
        print(formula_test)
        model = smf.logit(formula_test, data=join, missing='drop')
        try:
            results = model.fit()
        except:
            continue

    except (ValueError, TypeError):
        continue
    t.append((results.prsquared, name, results.pvalues[name]))
t.sort(reverse=True)
t

boy ~ caseid
Optimization terminated successfully.
         Current function value: 0.692996
         Iterations 3
boy ~ pregordr
Optimization terminated successfully.
         Current function value: 0.692962
         Iterations 3
boy ~ howpreg_n
boy ~ howpreg_p
boy ~ moscurrp
boy ~ nowprgdk
boy ~ pregend1
Optimization terminated successfully.
         Current function value: 0.692850
         Iterations 3
boy ~ pregend2
Optimization terminated successfully.
         Current function value: 0.528068
         Iterations 7
boy ~ nbrnaliv
Optimization terminated successfully.
         Current function value: 0.693001
         Iterations 3
boy ~ multbrth
Optimization terminated successfully.
         Current function value: 0.664221
         Iterations 5
boy ~ cmotpreg
boy ~ cmprgend
Optimization terminated successfully.
         Current function value: 0.692903
         Iterations 4
boy ~ flgdkmo1
boy ~ cmprgbeg
Optimization terminated successfully.
         Current function value: 0.692

/Users/nathankiner/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/nathankiner/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)



         Current function value: 0.378080
         Iterations: 35
boy ~ lpnctri
boy ~ workpreg
Optimization terminated successfully.
         Current function value: 0.692895
         Iterations 3
boy ~ workborn
Optimization terminated successfully.
         Current function value: 0.692553
         Iterations 3
boy ~ didwork
Optimization terminated successfully.
         Current function value: 0.692901
         Iterations 3
boy ~ matweeks
Optimization terminated successfully.
         Current function value: 0.691045
         Iterations 4
boy ~ weeksdk
Optimization terminated successfully.
         Current function value: 0.645648
         Iterations 5
boy ~ matleave
Optimization terminated successfully.
         Current function value: 0.692277
         Iterations 3
boy ~ matchfound
Optimization terminated successfully.
         Current function value: 0.692965
         Iterations 3
boy ~ livehere
Optimization terminated successfully.
         Current function value: 0.692132
     

/Users/nathankiner/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/nathankiner/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)



         Current function value: 0.600858
         Iterations: 35
boy ~ frsteatd_n2
Optimization terminated successfully.
         Current function value: 0.683487
         Iterations 4
boy ~ frsteatd_p2
Optimization terminated successfully.
         Current function value: 0.692440
         Iterations 3
boy ~ frsteatd2
Optimization terminated successfully.
         Current function value: 0.687620
         Iterations 4
boy ~ quitnurs2
Optimization terminated successfully.
         Current function value: 0.691185
         Iterations 3
boy ~ ageqtnur_n2
Optimization terminated successfully.
         Current function value: 0.657760
         Iterations 6
boy ~ ageqtnur_p2
Optimization terminated successfully.
         Current function value: 0.691843
         Iterations 4
boy ~ ageqtnur2
Optimization terminated successfully.
         Current function value: 0.658054
         Iterations 6
boy ~ matchfound3
Optimization terminated successfully.
         Current function value: 0.606620
 

...


         Current function value: 0.692967
         Iterations: 35
boy ~ fecund
Optimization terminated successfully.
         Current function value: 0.693011
         Iterations 3
boy ~ anybc36
Optimization terminated successfully.
         Current function value: 0.692987
         Iterations 3
boy ~ nosex36
Optimization terminated successfully.
         Current function value: 0.692978
         Iterations 3
boy ~ infert
Optimization terminated successfully.
         Current function value: 0.692964
         Iterations 3
boy ~ anybc12
Optimization terminated successfully.
         Current function value: 0.693010
         Iterations 3
boy ~ anymthd
Optimization terminated successfully.
         Current function value: 0.692999
         Iterations 3
boy ~ nosex12
Optimization terminated successfully.
         Current function value: 0.693005
         Iterations 3
boy ~ sexp3mo
Optimization terminated successfully.
         Current function value: 0.693018
         Iterations 3
boy ~ s

/Users/nathankiner/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/nathankiner/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)



         Current function value: 0.647741
         Iterations: 35
boy ~ sourcem4
boy ~ oldwp01
Optimization terminated successfully.
         Current function value: 0.693017
         Iterations 3
boy ~ oldwp02
Optimization terminated successfully.
         Current function value: 0.692902
         Iterations 3
boy ~ oldwp03
Optimization terminated successfully.
         Current function value: 0.692770
         Iterations 3
boy ~ oldwp04
Optimization terminated successfully.
         Current function value: 0.692616
         Iterations 3
boy ~ oldwp05
Optimization terminated successfully.
         Current function value: 0.692327
         Iterations 3
boy ~ oldwp06
Optimization terminated successfully.
         Current function value: 0.691414
         Iterations 3
boy ~ oldwp07
Optimization terminated successfully.
         Current function value: 0.691145
         Iterations 4
boy ~ oldwp08
Optimization terminated successfully.
         Current function value: 0.690012
         Ite

/Users/nathankiner/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


[(0.99999997478559399, u'cmfsexx20', 0.99487796794003025),
 (0.99999980810022737, u'cmfsexx14', 0.99044572447151147),
 (0.99999980810022737, u'cmfsexx12', 0.99044572447151147),
 (0.99999969668952193, u'cmvasrev', 0.98886355298197437),
 (0.99999943233443778, u'cmfsexx17', 0.98011303937835292),
 (0.99999943233443778, u'cmfsexx11', 0.98011303937835292),
 (0.99998725308812786, u'p1yhsage8', 0.94738623802804633),
 (0.99989411028244812, u'cmfsexx19', 0.84972483887354566),
 (0.99989411028244812, u'cmfsexx18', 0.84972483887354566),
 (0.99989411028244812, u'cmfsexx16', 0.84972483887354566),
 (0.9997238760876922, u'cmlstsxx17', 0.74192483418281008),
 (0.9997238760876922, u'cmlstsxx11', 0.74192483418281008),
 (0.9997238760876922, u'cmlsexx17', 0.74192483418281008),
 (0.9997238760876922, u'cmlsexx11', 0.74192483418281008),
 (0.76546223033783689, u'pcurrntx6', 0.99999450268546874),
 (0.76546223033553751, u'p1ycurrp6', 0.99988243132982202),
 (0.7123703286018046, u'mthusimx1727', 0.99705821865575273)

In [8]:
t

[(0.99999997478559399, u'cmfsexx20', 0.99487796794003025),
 (0.99999980810022737, u'cmfsexx14', 0.99044572447151147),
 (0.99999980810022737, u'cmfsexx12', 0.99044572447151147),
 (0.99999969668952193, u'cmvasrev', 0.98886355298197437),
 (0.99999943233443778, u'cmfsexx17', 0.98011303937835292),
 (0.99999943233443778, u'cmfsexx11', 0.98011303937835292),
 (0.99998725308812786, u'p1yhsage8', 0.94738623802804633),
 (0.99989411028244812, u'cmfsexx19', 0.84972483887354566),
 (0.99989411028244812, u'cmfsexx18', 0.84972483887354566),
 (0.99989411028244812, u'cmfsexx16', 0.84972483887354566),
 (0.9997238760876922, u'cmlstsxx17', 0.74192483418281008),
 (0.9997238760876922, u'cmlstsxx11', 0.74192483418281008),
 (0.9997238760876922, u'cmlsexx17', 0.74192483418281008),
 (0.9997238760876922, u'cmlsexx11', 0.74192483418281008),
 (0.76546223033783689, u'pcurrntx6', 0.99999450268546874),
 (0.76546223033553751, u'p1ycurrp6', 0.99988243132982202),
 (0.7123703286018046, u'mthusimx1727', 0.99705821865575273)

In [19]:
results = results.fit()

         Current function value: 0.199540
         Iterations: 35


/Users/nathankiner/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [13]:
SummarizeResults(results)

Intercept   -154   (1)
multbrth   -7.03   (1)
workpreg   6.55   (1)
methhist021   9.83   (1)
bc12pay111   14.8   (1)
ageprg02   0.00233   (0.655)
insuranc_r   48.4   (1)
R^2 0.7095


In [14]:
results.pvalues

Intercept      0.999999
multbrth       0.999995
workpreg       0.999928
methhist021    0.999867
bc12pay111     1.000000
ageprg02       0.655042
insuranc_r     1.000000
dtype: float64

In [35]:
results.pvalues['finalwgt_r']

0.99999999037299447

In [7]:
variables

[u'multbrth', u'workpreg', u'methhist021', u'bc12pay111', u'ageprg02']